In [185]:
import os
import torch
import pandas as pd
import numpy as np
from PIL import Image
from sklearn import svm
from sklearn.metrics import accuracy_score
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [2]:
pad_dataset = pd.read_csv('PAD dataset/metadata.csv')

In [3]:
pad_dataset.head()

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,...,NaN,NEV,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,5.0,BCC,TRUE,TRUE,FALSE,TRUE,TRUE,TRUE,PAT_46_881_939.png,True
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,...,NaN,ACK,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,PAT_1545_1867_547.png,False
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,...,NaN,ACK,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,...,5.0,BCC,TRUE,TRUE,FALSE,FALSE,TRUE,TRUE,PAT_684_1302_588.png,True


In [12]:
pd.set_option('display.max_columns', None)

In [13]:
pad_dataset.dropna(subset=['fitspatrick'])

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,cancer_history,has_piped_water,has_sewage_system,fitspatrick,region,diameter_1,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,True,True,True,3.0,NECK,6.0,5.0,BCC,TRUE,TRUE,FALSE,TRUE,TRUE,TRUE,PAT_46_881_939.png,True
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,False,False,False,1.0,FOREARM,5.0,5.0,BCC,TRUE,TRUE,FALSE,FALSE,TRUE,TRUE,PAT_684_1302_588.png,True
6,PAT_778,1471,False,True,GERMANY,ITALY,52,False,FEMALE,False,True,True,True,3.0,FACE,15.0,10.0,BCC,FALSE,TRUE,FALSE,TRUE,TRUE,TRUE,PAT_778_1471_835.png,True
7,PAT_117,179,False,False,POMERANIA,POMERANIA,74,True,FEMALE,False,False,False,False,1.0,FACE,15.0,10.0,BCC,TRUE,TRUE,TRUE,FALSE,TRUE,TRUE,PAT_117_179_983.png,True
9,PAT_705,4015,False,True,GERMANY,GERMANY,58,True,FEMALE,True,True,True,True,1.0,FOREARM,9.0,7.0,ACK,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,PAT_705_4015_413.png,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2289,PAT_386,785,True,False,POMERANIA,POMERANIA,66,False,MALE,True,False,True,True,2.0,EAR,20.0,15.0,ACK,TRUE,FALSE,TRUE,FALSE,TRUE,TRUE,PAT_386_785_536.png,True
2291,PAT_273,421,False,False,POMERANIA,POMERANIA,41,True,MALE,False,False,False,False,2.0,FACE,9.0,5.0,BCC,TRUE,UNK,TRUE,UNK,TRUE,TRUE,PAT_273_421_905.png,True
2292,PAT_491,934,False,False,POMERANIA,POMERANIA,43,True,FEMALE,True,True,False,False,2.0,FACE,9.0,5.0,SCC,TRUE,UNK,FALSE,UNK,TRUE,TRUE,PAT_491_934_46.png,True
2294,PAT_46,880,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,True,True,True,3.0,CHEST,13.0,12.0,BCC,TRUE,TRUE,FALSE,TRUE,FALSE,FALSE,PAT_46_880_140.png,True


In [ ]:
pad_dataset = pad_dataset[pad_dataset['diagnostic'] == 'MEL']

In [15]:
pad_dataset

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,cancer_history,has_piped_water,has_sewage_system,fitspatrick,region,diameter_1,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
53,PAT_680,1289,True,False,PORTUGAL,ITALY,78,False,MALE,True,True,True,True,2.0,BACK,10.0,10.0,MEL,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,PAT_680_1289_182.png,True
86,PAT_995,1867,False,False,UNK,UNK,49,False,FEMALE,False,False,False,False,2.0,THIGH,23.0,20.0,MEL,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,PAT_995_1867_5.png,True
112,PAT_966,1825,False,False,POMERANIA,POMERANIA,46,False,FEMALE,False,False,False,False,3.0,FACE,12.0,5.0,MEL,TRUE,TRUE,FALSE,TRUE,FALSE,FALSE,PAT_966_1825_584.png,True
164,PAT_966,1825,False,False,POMERANIA,POMERANIA,46,False,FEMALE,False,False,False,False,3.0,FACE,12.0,5.0,MEL,TRUE,TRUE,FALSE,TRUE,FALSE,FALSE,PAT_966_1825_546.png,True
262,PAT_495,941,False,False,POMERANIA,POMERANIA,58,True,MALE,False,True,True,True,2.0,BACK,14.0,11.0,MEL,FALSE,UNK,FALSE,TRUE,FALSE,FALSE,PAT_495_941_26.png,True
266,PAT_614,1166,False,False,UNK,UNK,56,False,FEMALE,False,True,True,True,3.0,CHEST,18.0,6.0,MEL,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,PAT_614_1166_885.png,True
281,PAT_340,714,False,True,ITALY,ITALY,54,False,FEMALE,False,True,True,True,4.0,BACK,14.0,12.0,MEL,TRUE,TRUE,FALSE,TRUE,FALSE,TRUE,PAT_340_714_68.png,True
296,PAT_115,1138,False,False,POMERANIA,POMERANIA,70,False,MALE,False,True,False,False,2.0,BACK,10.0,8.0,MEL,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,PAT_115_1138_870.png,True
338,PAT_895,1699,False,False,GERMANY,PORTUGAL,65,False,FEMALE,False,False,False,False,2.0,ARM,15.0,10.0,MEL,FALSE,UNK,FALSE,UNK,FALSE,FALSE,PAT_895_1699_872.png,True
412,PAT_233,354,False,False,POMERANIA,POMERANIA,58,True,FEMALE,True,True,False,False,2.0,ARM,9.0,4.0,MEL,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,PAT_233_354_935.png,True


In [18]:
pad_dataset["fitspatrick"].value_counts()

fitspatrick
2.0    35
3.0    10
4.0     4
1.0     3
Name: count, dtype: int64

In [ ]:
new_categories = {1: "light", 2: "light", 3: "medium", 4: "medium", 5: "dark", 6: "dark"}
pad_dataset["skin_tone_category"] = pad_dataset['fitspatrick'].map(new_categories)

In [42]:
pad_dataset["label"] = np.where(pad_dataset["diagnostic"] == "MEL", "melanoma", "not-melanoma")
pad_sorted = pad_dataset.sort_values(by="skin_tone_category")[["img_id", "skin_tone_category", "label"]]

In [43]:
pad_sorted

,img_id,skin_tone_category,label
53,PAT_680_1289_182.png,light,melanoma
2255,PAT_45_66_822.png,light,melanoma
1424,PAT_622_1185_922.png,light,melanoma
1461,PAT_656_1246_483.png,light,melanoma
1481,PAT_680_1289_585.png,light,melanoma
1575,PAT_611_1158_156.png,light,melanoma
1605,PAT_262_402_14.png,light,melanoma
1627,PAT_70_107_591.png,light,melanoma
1670,PAT_262_402_265.png,light,melanoma
1827,PAT_270_417_728.png,light,melanoma


In [44]:
metadata = pd.read_csv("sorted_metadata.csv")

In [48]:
metadata["img_id"] = metadata["DDI_file"]
metadata_clean = metadata[["skin_tone_category", "label", "img_id"]]

In [49]:
metadata_clean

,skin_tone_category,label,img_id
0,light,not-melanoma,000328.png
1,light,not-melanoma,000304.png
2,light,not-melanoma,000303.png
3,light,not-melanoma,000302.png
4,light,not-melanoma,000301.png
...,...,...,...
651,dark,not-melanoma,000167.png
652,dark,not-melanoma,000166.png
653,dark,not-melanoma,000165.png
654,dark,not-melanoma,000176.png


In [128]:
fitzpatrick17k = pd.read_csv("fitzpatrick17k_and_12missdata.csv")

In [130]:
three_dataset = pd.concat([metadata_clean, pad_sorted, fitzpatrick17k], ignore_index=True)

In [131]:
three_dataset = three_dataset.sort_values(by = "skin_tone_category")

In [132]:
three_dataset

,skin_tone_category,label,img_id
589,dark,not-melanoma,000234.png
583,dark,not-melanoma,000240.png
582,dark,not-melanoma,000241.png
581,dark,not-melanoma,000242.png
580,dark,not-melanoma,000245.png
...,...,...,...
436,medium,not-melanoma,000082.png
435,medium,not-melanoma,000184.png
434,medium,not-melanoma,000194.png
446,medium,not-melanoma,000655.png


In [197]:
three_dataset.to_csv('three_dataset.csv', index=False)

In [161]:
three_dataset["skin_tone_category"].value_counts()

skin_tone_category
dark      233
light     233
medium    233
Name: count, dtype: int64

In [145]:
pd.crosstab(three_dataset["skin_tone_category"], three_dataset["label"], margins=True)

label,melanoma,not-melanoma,All
skin_tone_category,,,
dark,33,200,233
light,33,200,233
medium,33,200,233
All,99,600,699


In [149]:
three_dataset[466:699]

,skin_tone_category,label,img_id
694,medium,melanoma,PAT_746_1407_708.png
745,medium,melanoma,2062.png
744,medium,melanoma,2164.png
743,medium,melanoma,2581.png
742,medium,melanoma,16534.png
...,...,...,...
436,medium,not-melanoma,000082.png
435,medium,not-melanoma,000184.png
434,medium,not-melanoma,000194.png
446,medium,not-melanoma,000655.png


In [ ]:
three_dataset_light = three_dataset[233:466]

In [150]:
three_dataset_medium = three_dataset[466:699]

In [151]:
three_dataset_dark = three_dataset[0:233]

In [152]:
import PIL

In [ ]:
def image_resize(image):
    original_size = image.size
    target_size = (224, 224)
    ratio = min(target_size[0] / original_size[0], target_size[1] / original_size[1])
    new_size = (int(original_size[0] * ratio), int(original_size[1] * ratio))
    resized_image = image.resize(new_size, Image.Resampling.LANCZOS)

    new_image = Image.new("RGB", target_size, (255, 255, 255))
    paste_position = ((target_size[0] - new_size[0]) // 2, (target_size[1] - new_size[1]) // 2)
    new_image.paste(resized_image, paste_position)

    return new_image

In [154]:
folder = "ddi + pad"
file_names = three_dataset_light["img_id"]
images = []
for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    # print(image.size) # Returns (width, height)
    images.append(np.array(image_resize(image)).flatten())
X_light = np.stack(images)

In [183]:
from efficientnet_pytorch import EfficientNet

In [184]:
model = EfficientNet.from_pretrained('efficientnet-b0')

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /Users/winnieye/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:01<00:00, 14.4MB/s]


Loaded pretrained weights for efficientnet-b0


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),       # Resize to 224x224
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],      # ImageNet means
        std=[0.229, 0.224, 0.225]        # ImageNet stds
    )
])

In [ ]:
img = Image.open('ddi + pad/000001.png')
transform = transforms.ToTensor()
tensor_image = transform(image).unsqueeze(0)
print(tensor_image.shape)

model.eval()
with torch.no_grad():
    logits = model(tensor_image)
preds = torch.topk(logits, k=5).indices.squeeze(0).tolist()
print(preds)

print('-----')
for idx in preds:
    label = "melanoma"
    prob = torch.softmax(logits, dim=1)[0, idx].item()
    print('{:<75} ({:.2f}%)'.format(label, prob*100))

torch.Size([1, 3, 1381, 999])
[78, 111, 885, 318, 310]
-----
melanoma                                                                    (5.41%)
melanoma                                                                    (3.44%)
melanoma                                                                    (2.03%)
melanoma                                                                    (1.06%)
melanoma                                                                    (0.98%)


In [155]:
from sklearn.model_selection import train_test_split

In [172]:
y_light = three_dataset_light["label"]

X_light_train, X_light_test, y_light_train, y_light_test = train_test_split(X_light, y_light, test_size=0.2, stratify=y_light, random_state=42)

clf_light = svm.SVC()

clf_light.fit(X_light_train, y_light_train)

y_light_pred = clf_light.predict(X_light_test)

accuracy_light = accuracy_score(y_light_test, y_light_pred)

print(accuracy_light)

0.9574468085106383


In [170]:
folder = "ddi + pad"
file_names = three_dataset_medium["img_id"]
images = []
for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    #print(image.size) # Returns (width, height)
    images.append(np.array(image_resize(image)).flatten())
X_medium = np.stack(images)

y_medium = three_dataset_medium["label"]

X_medium_train, X_medium_test, y_medium_train, y_medium_test = train_test_split(X_medium, y_medium, test_size=0.2, stratify=y_medium, random_state=42)

clf_medium = svm.SVC()

clf_medium.fit(X_medium_train, y_medium_train)

y_medium_pred = clf_medium.predict(X_medium_test)

accuracy_medium = accuracy_score(y_medium_test, y_medium_pred)

print(accuracy_medium)

0.8936170212765957


In [171]:
folder = "ddi + pad"
file_names = three_dataset_dark["img_id"]
images = []
for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    #print(image.size) # Returns (width, height)
    images.append(np.array(image_resize(image)).flatten())
X_dark = np.stack(images)

y_dark = three_dataset_dark["label"]

X_dark_train, X_dark_test, y_dark_train, y_dark_test = train_test_split(X_dark, y_dark, test_size=0.2, stratify=y_dark, random_state=42)

clf_dark = svm.SVC()

clf_dark.fit(X_medium_train, y_medium_train)

y_dark_pred = clf_dark.predict(X_dark_test)

accuracy_dark = accuracy_score(y_dark_test, y_dark_pred)

print(accuracy_dark)

0.851063829787234


In [173]:
folder = "ddi + pad"
file_names = three_dataset["img_id"]
images = []
for file in file_names:
    path = os.path.join(folder, file)
    image = PIL.Image.open(path)
    #print(image.size) # Returns (width, height)
    images.append(np.array(image_resize(image)).flatten())
X_total = np.stack(images)

y_total = three_dataset["label"]

X_total_train, X_total_test, y_total_train, y_total_test = train_test_split(X_total, y_total, test_size=0.2, random_state=42)

clf_total = svm.SVC()

clf_total.fit(X_total_train, y_total_train)

y_total_pred = clf_total.predict(X_total_test)

accuracy_total = accuracy_score(y_total_test, y_total_pred)

print(accuracy_total)

0.9


In [175]:
count_light = np.sum(y_light_pred == 'melanoma')
count_medium = np.sum(y_medium_pred == 'melanoma')
count_dark = np.sum(y_dark_pred == 'melanoma')
count_total = np.sum(y_total_pred == 'melanoma')
print(count_light)
print(count_medium)
print(count_dark)
print(count_total)

5
2
0
7


In [176]:
from sklearn.metrics import recall_score

In [177]:
recall_total = recall_score(y_total_test, y_total_pred, labels = ['melanoma', 'not-melanoma'], pos_label = 'melanoma')
print(recall_total)

0.3333333333333333


In [178]:
recall_light = recall_score(y_light_test, y_light_pred, labels = ['melanoma', 'not-melanoma'], pos_label = 'melanoma')
print(recall_light)

0.7142857142857143


In [179]:
recall_medium = recall_score(y_medium_test, y_medium_pred, labels = ['melanoma', 'not-melanoma'], pos_label = 'melanoma')
print(recall_medium)

0.2857142857142857


In [180]:
recall_dark = recall_score(y_dark_test, y_dark_pred, labels = ['melanoma', 'not-melanoma'], pos_label = 'melanoma')
print(recall_dark)

0.0
